In [2]:
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin

from shapely.geometry import mapping
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon

import matplotlib.pyplot as plt

In [6]:
#load data layers
f = '/Users/Zack/0_seawarden/python/detection/d1_targets/'
pts = gpd.read_file(f + 'greece_g25_100m_10-20_01-06.geojson')

f2 = '/Users/Zack/0_greece/'
search_area = gpd.read_file(f2 + 'AOI/mask_search_area_100m_5km.shp')
exclusion_zones = gpd.read_file(f2 + 'AOI/exclusions_2020-01-07.geojson')
farm_sites = gpd.read_file(f2 + 'blue_bridge/fishcages_Greece_farm_polygons_2.shp')

#re-project layers
pts = pts.to_crs({'init': 'epsg:2100'})
search_area = search_area.to_crs({'init': 'epsg:2100'})
exclusion_zones = exclusion_zones.to_crs({'init': 'epsg:2100'})

print('detections', len(pts), pts.crs)
print('search area', len(search_area), search_area.crs)
print('exclusion zones', len(exclusion_zones), exclusion_zones.crs)

detections 10041 {'init': 'epsg:2100'}
search area 1 {'init': 'epsg:2100'}
exclusion zones 15 {'init': 'epsg:2100'}


In [ ]:
# plt.rcParams['figure.figsize'] = (20, 10)
# ax=search_area.plot(linewidth=.5, edgecolor = 'black', facecolor = 'none')
# pts.plot(markersize=5, facecolor='red', ax=ax)
# exclusion_zones.plot(ax=ax)

In [ ]:
%%time 
#clip points to search area
clip = sjoin(pts, search_area, how='inner', op='within')
print('detections within search area:', len(clip))

In [ ]:
#plt.rcParams['figure.figsize'] = (20, 10)
#ax=search_area.plot(linewidth=.5, edgecolor = 'black', facecolor = 'none')
#clip.plot(markersize=5, facecolor = 'red', ax=ax)

In [ ]:
%%time 
#combine all exclusion zones
exclusion_zones['Dissolve'] = 0
exclusion_zones_dis = exclusion_zones.dissolve(by='Dissolve')

In [ ]:
%%time 
#exclude points in exclusion areas
mask = ~clip.within(exclusion_zones_dis.loc[0, 'geometry'])
clip2 = clip.loc[mask]
print('detections outside of exclusion zones:', len(clip2))

In [ ]:
#ax=search_area.plot(linewidth=.5, edgecolor = 'black', facecolor = 'none')
#exclusion_zones_dis.plot(facecolor = 'blue', ax=ax)
#clip2.plot(markersize=5, facecolor = 'red', ax=ax)

In [ ]:
%%time 
#buffer points, dissolve buffers to aggregate points near each other
buffer = gpd.GeoDataFrame(geometry = clip2.buffer(10))
buffer['Dissolve'] = 0
buffer_dis = buffer.dissolve(by='Dissolve')

In [ ]:
#explode function
def explode(indf):
    outdf = gpd.GeoDataFrame(columns=indf.columns)
    for idx, row in indf.iterrows():
        if type(row.geometry) == Polygon:
            outdf = outdf.append(row,ignore_index=True)
        if type(row.geometry) == MultiPolygon:
            multdf = gpd.GeoDataFrame(columns=indf.columns)
            recs = len(row.geometry)
            multdf = multdf.append([row]*recs,ignore_index=True)
            for geom in range(recs):
                multdf.loc[geom,'geometry'] = row.geometry[geom]
            outdf = outdf.append(multdf,ignore_index=True)
    return outdf

In [ ]:
%%time 
#explode polygon and generate centroids
buffer_exploded = explode(buffer_dis)      
centroids = gpd.GeoDataFrame(geometry = buffer_exploded.centroid)
print('detections after aggregation:', len(centroids))

In [ ]:
%%time 
#buffer centroids and make square polygons
centroid_buffer = gpd.GeoDataFrame(geometry = centroids.buffer(50))
envelope = gpd.GeoDataFrame(geometry = centroid_buffer.envelope)

In [ ]:
#plt.rcParams['figure.figsize'] = (20, 10)
#ax=search_area.plot(linewidth=.5, edgecolor = 'black', facecolor = 'none')
#centroid_buffer.plot(markersize=5, facecolor = 'black', ax=ax)
#envelope.plot(linewidth=.5, edgecolor = 'black', facecolor = 'none', ax=ax)

In [ ]:
#plt.rcParams['figure.figsize'] = (20, 10)
#ax = centroids.loc[[0], 'geometry'].plot(markersize=10, facecolor = 'black')
#centroid_buffer.loc[[0], 'geometry'].plot(linewidth=.5, edgecolor = 'black', facecolor = 'none', ax=ax)
#envelope.loc[[0], 'geometry'].plot(linewidth=.5, edgecolor = 'black', facecolor = 'none', ax=ax)

In [ ]:
#set crs
envelope.crs = {'init' :'epsg:2100'}
envelope['geometry'] = envelope['geometry'].to_crs(epsg=4326)
print(envelope.crs)
envelope.head()

In [ ]:
#extract lat/long for each square polygon (envelope)
coord_list = []
for i in range(len(envelope)):
    coords = mapping(envelope.geometry[i])['coordinates']
    coord_list.append(coords)

In [ ]:
#function to extract and format x/y points
def coord(input_list):
    pt1, pt2, pt3, pt4, pt5 = map(list, zip(*input_list))
    x1, y1, = map(list, zip(*pt1))
    x2, y2, = map(list, zip(*pt2))
    x3, y3, = map(list, zip(*pt3))
    x4, y4, = map(list, zip(*pt4))
    x5, y5, = map(list, zip(*pt5))

    x1=str(x1).strip("[]")
    x2=str(x2).strip("[]")
    x3=str(x3).strip("[]")
    x4=str(x4).strip("[]")
    x5=str(x5).strip("[]")

    y1=str(y1).strip("[]")
    y2=str(y2).strip("[]")
    y3=str(y3).strip("[]")
    y4=str(y4).strip("[]")
    y5=str(y5).strip("[]")

    coord_group = {'x': [x1, x2, x3, x4, x5], 'y': [y1, y2, y3, y4, y5]}
    coord_group_df = pd.DataFrame(data=coord_group)
    return coord_group_df

In [ ]:
# #combine x/y point groups
coord_all = []
for i in range(len(coord_list)):
    coord_group = coord(coord_list[i])
    coord_group['id']=i
    coord_all.append(coord_group)
    
targets = pd.concat(coord_all)
print('total coordinates (4 per detection):', len(targets))
targets.head()

In [ ]:
#accuracy check
centroids_test = centroids.copy()
centroids_test.crs = {'init' :'epsg:2100'}
centroids_test['geometry'] = centroids_test['geometry'].to_crs(epsg=4326)

matches = centroids_test.intersects(farm_sites.unary_union)
count = centroids_test.loc[matches]

detections_n = len(centroids_test)
farm_n = len(farm_sites)
matches_n = len(count)

print('total detections:', len(pts))
print('detections in search area:', detections_n)
print('farm sites:', farm_n)
print('matches:', matches_n)
print('positive matches', round(matches_n / farm_n, 3))
print('overall accuracy', round(matches_n / detections_n, 3))

#plt.rcParams['figure.figsize'] = (20, 10)
#ax=farm_sites.plot(linewidth=.5, edgecolor = 'black', facecolor = 'none')
#centroids_test.plot(markersize=1, facecolor = 'red', ax=ax)

In [ ]:
# pts.to_file(f + 'd1_sar/g25_100m_10-20_01-06.shp')
# targets.to_csv(f + 'd1_sar/g25_100m_10-20_01-06_v2.csv', index = None, header=True)

# centroids_test.crs = {'init' :'epsg:4326'}
# centroids_test.to_file(f + 'd1_sar/g25_100m_10-20_01-06_v2.shp')
# centroids_test.to_file(f + 'd1_sar/g25_100m_10-20_01-06_v2.geojson', driver='GeoJSON')